In [1]:
import sys
import os

notebook_dir = os.getcwd()
parent_dir = os.path.dirname(notebook_dir)
sys.path.append(parent_dir)

from src.utils.results_utils import *
from src.utils.data_utils import str_dict_to_values
from src.utils.ml_utils import *
import pandas as pd
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import pickle
import ast

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\amaur\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package names to
[nltk_data]     C:\Users\amaur\AppData\Roaming\nltk_data...
[nltk_data]   Package names is already up-to-date!


## Load and clean the dataset

In [2]:
file_name = 'data/cleaned.csv'

for path in sys.path:
    file_path = os.path.join(path, file_name)

df_ml = pd.read_csv(file_path)
df_ml['Country'] = df_ml['Country'].apply(str_dict_to_values)

In [3]:
df_ml.drop(columns=['Wikipedia_ID','Name', 'Sex', 'Actor_age', 'Country','Release_date', 'Languages'],inplace=True)

df_ml.head()

,Character_name,Genre_Category
0,Akooshay,"['Action & Adventure', 'Horror & Thriller', 'F..."
1,Melanie,"['Action & Adventure', 'Horror & Thriller', 'F..."
2,Williams,"['Action & Adventure', 'Horror & Thriller', 'F..."
3,Jericho,"['Action & Adventure', 'Horror & Thriller', 'F..."
4,Bashira,"['Action & Adventure', 'Horror & Thriller', 'F..."


## Prepare the data for the model with the different features

In [4]:
# Encoding of the genre names
genres_list = ['Action & Adventure', 'Drama', 'Comedy', 'Horror & Thriller', 
              'Fantasy & Sci-Fi', 'Historical & War', 'Romance', 'Documentary', 
              'Music & Performance', 'Cult & B-Movies', 'Other']
df_ml['Genre_Category'] = df_ml['Genre_Category'].apply(ast.literal_eval)

mlb = MultiLabelBinarizer(classes=genres_list)
genre_encoded = mlb.fit_transform(df_ml['Genre_Category'])

genre_df = pd.DataFrame(genre_encoded, columns=mlb.classes_)

In [5]:
# Add the encoded genre in the DataFrame
df_ml = pd.concat([df_ml['Character_name'], genre_df], axis=1)

df_ml = df_ml.reset_index(drop=True)
df_ml.sample(5)

,Character_name,Action & Adventure,Drama,Comedy,Horror & Thriller,Fantasy & Sci-Fi,Historical & War,Romance,Documentary,Music & Performance,Cult & B-Movies,Other
121102,Richardson,0,1,1,0,0,0,0,0,0,0,0
75507,Edgtho,1,0,0,0,1,0,0,0,0,0,0
144844,Darl,1,1,0,1,0,0,0,0,0,0,0
89736,Jen,1,0,1,1,1,0,0,0,0,0,0
107701,Emil,1,0,0,1,0,1,0,0,0,0,0


We will add the most frequent special character to the alphabet: 'é', 'è', 'á' and 'í'

In [6]:
augmented_alphabet = 'abcdefghijklmnopqrstuvwxyzéèíá'

In [7]:
# Add features to the df
character_processor = NameFeatureProcessor('Character_name', ngram_range = (2,2))

df_ml = character_processor.process(df_ml,alphabet = augmented_alphabet, analyze_name = True, diacritic = False, phonetics = False, first_last = True, ngram=False)
df_ml.head()

,Character_name,Action & Adventure,Drama,Comedy,Horror & Thriller,Fantasy & Sci-Fi,Historical & War,Romance,Documentary,Music & Performance,...,u_l,v_l,w_l,x_l,y_l,z_l,é_l,è_l,í_l,á_l
0,Akooshay,1,0,0,1,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,Melanie,1,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Williams,1,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Jericho,1,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Bashira,1,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
# Add ngrams using HashingVectorizer with 100 features
vectorizer = HashingVectorizer(analyzer='char', ngram_range=(2, 3), n_features=300)
ngram_features = vectorizer.fit_transform(df_ml['Character_name'])
n_gram_df = pd.DataFrame(ngram_features.toarray())
df_ml = pd.concat([df_ml, n_gram_df], axis=1)

In [9]:
with open('hashing_vectorizer_genre.pkl', 'wb') as f:
        pickle.dump(vectorizer, f)

In [10]:
df_ml.sample(10)

,Character_name,Action & Adventure,Drama,Comedy,Horror & Thriller,Fantasy & Sci-Fi,Historical & War,Romance,Documentary,Music & Performance,...,290,291,292,293,294,295,296,297,298,299
30456,Schmuck,1,0,0,1,1,0,0,0,0,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0
89577,Venus,0,1,1,0,1,0,1,0,1,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,-0.377964,0.0,0.000000,0.0
121685,Snuf,0,1,0,0,0,0,0,0,0,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.447214,0.0,0.000000,0.0
117848,Viovanni,1,0,0,0,0,0,0,0,0,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0
10236,Maureen,0,0,1,0,0,0,1,0,0,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0
17061,Halloran,1,1,0,1,0,0,0,0,0,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0
38853,Jen,0,0,1,0,0,0,0,0,0,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0
70528,Elspeth,0,1,1,0,1,0,0,0,0,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,-0.301511,0.0
29138,Laurie,1,1,0,1,0,1,0,0,0,...,0.0,0.000000,0.000000,0.0,0.333333,0.0,0.000000,0.0,0.000000,0.0
144693,Nottingham,0,1,1,0,1,0,0,0,0,...,0.0,-0.242536,-0.242536,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0


## Genre Prediction Model

In [11]:
df_ml_genres = df_ml.copy()
df_ml_genres.columns = df_ml_genres.columns.astype(str)

In [12]:
rows_with_nans = df_ml_genres.isna().any(axis=1).sum()
print(rows_with_nans)

0


In [13]:
# Drop potential NaN values
df_ml_genres = df_ml_genres.dropna()

In [14]:
# Define X and target Y
X = df_ml_genres.drop(['Character_name', 'Action & Adventure', 'Drama', 'Comedy',
             'Horror & Thriller', 'Fantasy & Sci-Fi', 'Historical & War', 'Romance',
             'Documentary', 'Music & Performance', 'Cult & B-Movies', 'Other'], axis=1)
y = df_ml_genres[['Action & Adventure', 'Drama', 'Comedy', 'Horror & Thriller', 'Fantasy & Sci-Fi',
        'Historical & War', 'Romance', 'Documentary', 'Music & Performance', 'Cult & B-Movies', 'Other']]

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [16]:
# Use Random Forest model
clf = MultiOutputClassifier(RandomForestClassifier(n_estimators=50, max_depth=10, class_weight="balanced", random_state=42))
clf.fit(X_train, y_train)

MultiOutputClassifier(estimator=RandomForestClassifier(class_weight='balanced',
                                                       max_depth=10,
                                                       n_estimators=50,
                                                       random_state=42))

In [17]:
y_pred = clf.predict(X_test)

In [18]:
print(classification_report(y_test, y_pred, target_names=y.columns))

                     precision    recall  f1-score   support

 Action & Adventure       0.44      0.60      0.51      6457
              Drama       0.66      0.62      0.64     10232
             Comedy       0.44      0.43      0.43      6280
  Horror & Thriller       0.28      0.70      0.40      4009
   Fantasy & Sci-Fi       0.15      0.49      0.23      2010
   Historical & War       0.13      0.49      0.20      1615
            Romance       0.25      0.52      0.33      3512
        Documentary       0.04      0.50      0.08       523
Music & Performance       0.14      0.43      0.21      1234
    Cult & B-Movies       0.15      0.64      0.24      1920
              Other       0.05      0.44      0.09       404

          micro avg       0.28      0.56      0.38     38196
          macro avg       0.25      0.53      0.31     38196
       weighted avg       0.40      0.56      0.44     38196
        samples avg       0.29      0.56      0.36     38196



c:\Users\amaur\anaconda3\envs\ada\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [19]:
# Save the model
with open('model_genres.pkl', 'wb') as f:
    pickle.dump(clf, f)

## Prediction for a chosen name

In [20]:
def feature_creation(name):
    df_pred = pd.DataFrame([name], columns=['Name'])
    pred_processor = NameFeatureProcessor('Name', ngram_range=(2,2))
    df_pred = pred_processor.process(df_pred, alphabet=augmented_alphabet, analyze_name=True, diacritic=False, phonetics=False, first_last=True, ngram=False)
    
    # Load pre-trained HashingVectorizer
    with open('hashing_vectorizer_genre.pkl', 'rb') as f:
        vectorizer = pickle.load(f)
    
    # Transform the name using the vectorizer
    ngram_name = vectorizer.transform(df_pred['Name'])
    ngram_name_df = pd.DataFrame(ngram_name.toarray())
    df_pred = pd.concat([df_pred, ngram_name_df], axis=1)
    
    return df_pred

def predict(df):
    df.drop(columns=['Name'], inplace=True)
    df.columns = df.columns.astype(str)  # Ensure column names are strings
    return clf.predict(df)

def create_and_predict(name):
    df = feature_creation(name)
    pred = predict(df)
    
    # Decode the binary prediction results to genre names
    decoded_genres = mlb.inverse_transform(pred)

    print(decoded_genres)

In [21]:
create_and_predict('Fanny')

[('Action & Adventure', 'Drama', 'Comedy', 'Romance', 'Documentary', 'Cult & B-Movies')]
